# A simple RAG application using open-source models

In [55]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
MODEL = "mixtral:8x7b"
MODEL = "llama2"


In [56]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

"\nSure, here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope you found that amusing. If you'd like to hear another one, just let me know!"

In [57]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

"Sure! Here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope you found that amusing. Let me know if you'd like to hear another joke!"

In [58]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [59]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

' Sure! Based on the context you provided, my answer would be:\n\n"My name is Santiago."'

In [60]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("mlschool.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='3/7/24, 2:37 PM Building Machine Learning Systems That Don\'t Suck\nhttps://www.ml.school 1/15Building Machine Learning Systems That Don\'t\nSuck\nA live, interactive program that\'ll help you build production-ready\nmachine learning systems from the ground up.\nNext cohort:\xa0April8 - 25, 2024\nCheck the schedulefor more details about upcoming cohorts.\nRegister now\nLearn how to design, build, deploy, and scale machine learning\nsystems to solve real-world problems."This is the best machine learning course I\'ve done\never. Worth every cent."\n— Jose Reyes, AI/ML at Cevo Australia', metadata={'source': 'mlschool.pdf', 'page': 0}),
 Document(page_content="3/7/24, 2:37 PM Building Machine Learning Systems That Don't Suck\nhttps://www.ml.school 2/15I'll lose my mind if I see another book or course teaching people the same basic\nideas for the hundredth time. Most people are stuck in beginner mode, and\nfinding help to solve real-world problems is hard.\nI want t

In [61]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [62]:
retriever = vectorstore.as_retriever()
retriever.invoke("machine learning")

[Document(page_content='3/7/24, 2:37 PM Building Machine Learning Systems That Don\'t Suck\nhttps://www.ml.school 4/15wants to use machine learning to solve real-world problems.\nHere are the criteria to succeed in the program:\nYou have experience writing code. We\'ll use Python throughout the class,\nbut you won\'t have any problems if you know any other language.\nYou are familiar with basic machine learning terminology. This is not an\nintroductory class. We\'ll move quickly over the basics to focus on the\nfundamental ideas that make systems work.\nYou are ready to put in the work to succeed.\n"I have learned a ton from Santiago in his class and it was actually what helped\ninspire me and get into the MLOps work that I\'m doing now. Truly one of the\nmost helpful online courses for doing real, full-scale machine learning."\nBrian H. Hough\nSoftware Engineer', metadata={'source': 'mlschool.pdf', 'page': 3}),
 Document(page_content="3/7/24, 2:37 PM Building Machine Learning Systems 

In [63]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [64]:
questions = [
    "What is the purpose of the course?",
    "How many hours of live sessions?",
    "How many coding assignments are there in the program?",
    "Is there a program certificate upon completion?",
    "What programming language will be used in the program?",
    "How much does the program cost?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the purpose of the course?
Answer: Based on the context provided, the purpose of the course appears to be to teach students how to build machine learning systems that work in the real world. The course aims to provide practical experience and hands-on training, rather than just teaching basic ideas or theories. The instructor wants to help students unlearn what they think machine learning is and instead focus on building end-to-end systems from scratch. The course offers lifetime access to live sessions, coding instructions, and final projects, with the goal of providing direct feedback on students' work.

Question: How many hours of live sessions?
Answer: Based on the context provided, the program includes 18 hours of live, interactive sessions.

Question: How many coding assignments are there in the program?
Answer: Based on the context provided, there are 30 coding assignments and practice questions in the program.

Question: Is there a program certificate upon com

In [54]:
chain.batch([{"question": q} for q in questions])

[' Based on the provided document, the purpose of the course "Building Machine Learning Systems That Don\'t Suck" is to provide students with practical skills and insights into building machine learning systems that work in real-world scenarios. The course focuses on designing and writing code to build an end-to-end machine learning system from scratch and covers best practices for tackling challenges faced by machine learning engineers. The course also teaches how to use various techniques like active learning, distributed training, adversarial validation, human-in-the-loop deployments, model compression, test-time augmentation, testing in production, among others. Additionally, the course covers creating training, deploying, monitoring, and inference pipelines using Amazon SageMaker and open-source tools. The course emphasizes learning from real-life examples instead of theoretical concepts.',
 ' Based on the documents provided, there are 18 hours of live, interactive sessions in the

In [30]:
for s in chain.stream({"question": "What is the purpose of the course?"}):
    print(s, end="", flush=True)

The purpose of the course is to teach practical skills and insights that will help individuals build machine learning systems that work in the real world.

In [109]:
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# loader = WebBaseLoader("https://www.ml.school")
# docs = loader.load()
# documents = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=200
# ).split_documents(docs)

# documents

[Document(page_content='Building Machine Learning Systems That Don\'t Suck"This is the best machine learning course I\'ve done ever. Worth every cent."Jose Reyes, AI/ML at Cevo AustraliaBuilding Machine Learning Systems That Don\'t SuckA live, interactive program that\'ll help you build production-ready machine learning systems from the ground up.Next cohort:\xa0April 8 - 25, 2024Check the schedule for more details about upcoming cohorts.Register nowLearn how to design, build, deploy, and scale machine learning systems to solve real-world problems.I\'ll lose my mind if I see another book or course teaching people the same basic ideas for the hundredth time. Most people are stuck in beginner mode, and finding help to solve real-world problems is hard.I want to change that.I started writing software 30 years ago. I\'ve written pipelines and trained models for some of the largest companies in the world. I want to show you how to do the same.This is the class I wish I had taken when I star